In [1]:
import numpy as np
import scipy.misc
import scipy.io
import matplotlib.pyplot as plt
%matplotlib inline
from functools import lru_cache

In [46]:
class Labeler(object):
    """Assigns unique integer labels."""
    def __init__(self, init=()):
        self._labels = {}
        self._xs = []
        for x in init:
            self.label(x)
    
    def __contains__(self, x):
        return x in self._labels
    
    def __len__(self):
        return len(self._labels)

    def label(self, x):
        if x not in self._labels:
            self._labels[x] = len(self._labels)
            self._xs.append(x)
        return self._labels[x]

    def unlabel(self, label):
        return self._xs[label]

    __call__ = label

In [53]:
initial_state = ((0,0), (0,0), (0, 0))  # (alpha, beta) for each arm
final_state = label(0)

n_arm = 2
n_action = n_arm + 1
cost = 0.01
term_action = n_arm

T = defaultdict(lambda: defaultdict(lambda: np.zeros(n_action))) # s, s1, a
R = defaultdict(lambda: np.zeros(n_action)) # s, a

label = Labeler()

def outcomes(s, a):
    alpha, beta = state[a]
    s1 = state[:a] + ((alpha + obs, beta + 1 - obs),) + state[a+1:]
    return sorted(s1)  # state space reduction

def visit(s):
    if label(s) in T:  # already done
        return
    for a in range(n_arm):
        for prob, s1 in outcomes(s, a):       
            T[label(s)][label(s1)][a] = prob
            R[label(s)][a] = cost
            visit(s1)
    T[label(s)][final_state][a] = 1
    R[label(s)][term_action] = -1  # expectation of best arm
    
visit(initial_state)

In [48]:
n_state = len(label)
T1 = np.zeros((n_state, n_state, n_action))
R1 = np.zeros((n_state, n_action))
# Move values from T and R into T1 and R1

TypeError: object of type 'Labeler' has no len()